# Dynamic
Пример отчета с динамикой аудитории интернет-проекта.

## Описание задачи

Посчитаем аудиторию проекта Kommersant.ru, а также аудиторию всего Интернета в динамике за 6 месяцев с января по июнь 2020 года.

Общие параметры:
- Период: 1 января - 30 июня 2020 года
- География: Россия 0+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline).

Статистики:
- Unweighted Reach (UnwReach)
- Reach
- Reach % (ReachPer)
- Average Daily Reach (ADR)
- Average Daily Reach % (ADRPer)
- Average Weekly Reach (AWR)
- Average Weekly Reach % (AWRPer)


## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot

import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool

import logging

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

## Формируем задания

Необходимо сформировать два задания для Responsum:
- расчет динамики аудитории проекта Kommersant.ru
- расчет динамики аудитории Total Internet

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile, т.к. в расчетах участвуют все типы устройств (Web Desktop, Web Mobile, App Online, App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-01-01'
date_to = '2020-06-30'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Получим ID проектов
Для построения отчета нам необходимо получить идентификатор __Kommersant.ru__, для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Kommersant.ru  site_id = 118

## Задания
Перейдем к формированию заданий.

### Задание №1. Аудитория проекта Kommersant.ru в динамике


In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'kommersant'

# задаем медиа фильтр, в нашем случае это ID проекта kommersant.ru
media_filter = "(site = 118)"

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# указываем порядок группировки по месяцам
structure =  {
    "usetype": False,
    "date": "month"
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_project = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_project

### Задание №2. Аудитория всего Интернета в динамике

Расчет данных за полгода по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.

Получим код технической страницы WEB-Index

In [ ]:
rcats.find_media('Техническая страница', find_in=['site'])

Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total_dynamic'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = "(site = 101)"

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ADR", "AWR", "ReachPer", "ADRPer", "AWRPer"]

# указываем порядок группировки по месяцам
structure =  {
    "usetype": False,
    "date": "month"
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total_dynamic = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total_dynamic

### Формирование итоговой таблицы

Следующим шагом соберем полученные результаты в один DataFrame

In [ ]:
# Объединим наши DataFrame'ы
df_result = pd.concat([df_project, df_total_dynamic])
df_result

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('kommersant-dynamic'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()